<a id="Index"></a>
## Index
Run 0 and 1, and then each cell is independent! :)

0.[Setup](#Setup)

1.[Load Data](#LoadData)

2.[Simple OLS](#OLS)

3.[Gradients by hand](#grad)

4.[Complete without Utilities](#complete)

5.[Complete with Utilities](#completeWUtilities)

<a id="Setup"></a>
## 0. Setup
[Back to Index](#Index)

In [1]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

<a id="LoadData"></a>
## 1. Load Data
[Back to Index](#Index)

In [2]:
n=1000
m=2
XX=np.random.rand(n,m)
eps=np.random.rand(n,1)/10
beta=[[3],[2]]
YY=np.matmul(XX,beta)+eps


<a id="OLS"></a>

## 2. OLS with TF
[Back to Index](#Index)

In [24]:
X = tf.constant(XX, dtype=tf.float32, name="X")
y = tf.constant(YY, dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

with tf.Session() as sess:
    print("theta:", theta.eval())
    print("mse:", mse.eval())    





theta: [[ 3.04084778]
 [ 2.04269505]]
mse: 0.00114775





<a id="grad"></a>

## 3. Gradient by hand
[Back to Index](#Index)

In [25]:
n_epochs = 1000
learning_rate = 0.01


X = tf.constant(XX, dtype=tf.float32, name="X")
y = tf.constant(YY, dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([m, 1], -1.0, 1.0), name="theta")



y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/n * tf.matmul(tf.transpose(X), error)
gradients = tf.gradients(mse, [theta])[0]

training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)


    for epoch in range(n_epochs):
        sess.run(training_op)

    best_theta = theta.eval()
    best_mse=mse.eval()

print(best_theta)
print(best_mse)

[[ 3.11065578]
 [ 1.97183597]]
0.00194272



<a id="complete"></a>

## 4. Complete Version (No utilities)
[Back to Index](#Index)

In [3]:
# Load Data First
n_epochs = 1000
batch_size = 10
n_batches = int(np.ceil(n / batch_size))
learning_rate = 0.01
momentum_rate=0.9



# Graph (We will evaluate we n rows but also batch rows, so we use None to be flexible)
X=tf.placeholder(tf.float32, shape=(None,m),name="X")
y=tf.placeholder(tf.float32, shape=(None,1),name="y")

theta = tf.Variable(tf.random_uniform([m, 1], -1.0, 1.0), name="theta")

y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

init = tf.global_variables_initializer()
optimizer=tf.train.GradientDescentOptimizer(learning_rate= learning_rate)
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=momentum_rate)
training_op=optimizer.minimize(mse)


def fetch_batch(epoch, batch, batch_size):
    np.random.seed(epoch * n_batches + batch)
    indices = np.random.randint(m, size=batch_size)  
    X_batch = XX[indices] # not shown
    y_batch = YY[indices] # not shown
    return X_batch, y_batch



#Execution Phase

with tf.Session() as sess:
    sess.run(init)    
    for epoch in range(n_epochs):
        for batch in range(n_batches):
            x_batch,y_batch=fetch_batch(epoch, batch, batch_size)
            sess.run(training_op,
                     feed_dict={X:x_batch, y:y_batch})
    print(theta.eval())
    

            


[[ 3.0243175 ]
 [ 2.09133148]]



<a id="completeWUtilities"></a>


## Complete Version (W/ utilities)
[Back to Index](#Index)

In [5]:
# Load Data First
n_epochs = 100
batch_size = 10
n_batches = int(np.ceil(n / batch_size))
learning_rate = 0.01
momentum_rate=0.9



def fetch_batch(epoch, batch, batch_size):
    np.random.seed(epoch * n_batches + batch)
    indices = np.random.randint(m, size=batch_size)  
    X_batch = XX[indices] # not shown
    y_batch = YY[indices] # not shown
    return X_batch, y_batch


# Graph (We will evaluate we n rows but also batch rows, so we use None to be flexible)
tf.reset_default_graph()


X=tf.placeholder(tf.float32, shape=(None,m),name="X")
y=tf.placeholder(tf.float32, shape=(None,1),name="y")

theta = tf.Variable(tf.random_uniform([m, 1], -1.0, 1.0), name="theta")

y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

init = tf.global_variables_initializer()
optimizer=tf.train.GradientDescentOptimizer(learning_rate= learning_rate)
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=momentum_rate)
training_op=optimizer.minimize(mse)


mse_summary=tf.summary.scalar('MSE',mse)
file_writer=tf.summary.FileWriter('./logdir', tf.get_default_graph())


#Execution Phase

with tf.Session() as sess:
    sess.run(init)    
    for epoch in range(n_epochs):
        for batch in range(n_batches):
            x_batch,y_batch=fetch_batch(epoch, batch, batch_size)
            summary_str = mse_summary.eval(feed_dict={X: x_batch, y: y_batch})
            step = epoch * n_batches + batch
            file_writer.add_summary(summary_str,step)
            sess.run(training_op,feed_dict={X:x_batch, y:y_batch})
            
            

    
    print(theta.eval())
    
file_writer.close()

[[ 2.94111276]
 [ 2.18228817]]


In [6]:
! tensorboard --logdir ./logdir

Starting TensorBoard 41 on port 6006
(You can navigate to http://172.17.0.2:6006)
^C
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 11, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/tensorboard/tensorboard.py", line 151, in main
    tb_server.serve_forever()
  File "/usr/lib/python2.7/SocketServer.py", line 236, in serve_forever
    poll_interval)
  File "/usr/lib/python2.7/SocketServer.py", line 155, in _eintr_retry
    return func(*args)
KeyboardInterrupt
